In [14]:
import os
import re
import csv

def extract_conferences_from_aggregate(file_path):
    """
    Trích xuất thông tin hội nghị từ file shorten_aggregate_file.txt.
    Trả về dictionary với key là tên file và value là nội dung.
    """
    with open(file_path, 'r', encoding='utf-8') as file:
        data = file.read()
    conferences = {}
    matches = re.findall(r"File:\s+(.*?)\n(.*?)(?=File:\s+|\Z)", data, re.S)
    for match in matches:
        file_name, content = match
        conferences[file_name.strip()] = content.strip()
    return conferences

def find_conference_in_responses(conference_name, responses_folder):
    """
    Tìm thông tin hội nghị trong các file của thư mục responses.
    """
    for response_file in os.listdir(responses_folder):
        response_file_path = os.path.join(responses_folder, response_file)
        with open(response_file_path, 'r', encoding='utf-8') as file:
            data = file.read()

        # Tìm kiếm thông tin liên quan đến hội nghị
        match = re.search(rf'\d+\.\s+{conference_name} information:\s+(.*?)(?=\d+\.\s+|\Z)', data, re.S)
        if match:
            return match.group(1).strip()  # Trả về nội dung hội nghị
    return None

def main(aggregate_file, responses_folder, output_csv):
    # Đọc thông tin hội nghị từ shorten_aggregate_file.txt
    response_data = extract_conferences_from_aggregate(aggregate_file)
    
    results = []

    for file_name, response_content in response_data.items():
        # Tên hội nghị là phần tên file không bao gồm đuôi .txt
        conference_name = file_name.replace('.txt', '').strip()
        response_info = find_conference_in_responses(conference_name, responses_folder)
        if response_info:
            results.append({'input': response_content, 'output': response_info})

    # Lưu kết quả vào file CSV
    with open(output_csv, 'w', newline='', encoding='utf-8') as csvfile:
        writer = csv.DictWriter(csvfile, fieldnames=['input', 'output'])
        writer.writeheader()
        writer.writerows(results)

if __name__ == "__main__":
    aggregate_file = "./shorten_aggregate_file.txt"  # Đường dẫn file shorten_aggregate_file.txt
    responses_folder = "./responses"                # Thư mục chứa các file responses
    output_csv = "./train_data.csv"                 # Đường dẫn file CSV kết quả
    main(aggregate_file, responses_folder, output_csv)


In [17]:
import csv
import sys

def validate_csv_structure_and_count(csv_file_path):
    """
    Kiểm tra cấu trúc của file CSV và đếm số lượng input, output hợp lệ.
    
    Args:
        csv_file_path (str): Đường dẫn tới file CSV.
        
    Returns:
        dict: Kết quả kiểm tra và số lượng input, output hợp lệ.
    """
    result = {"is_valid": True, "errors": [], "input_count": 0, "output_count": 0}
    
    try:
        # Tăng giới hạn kích thước của một dòng CSV
        max_int = sys.maxsize
        while True:
            try:
                csv.field_size_limit(max_int)
                break
            except OverflowError:
                max_int = max_int // 2
        
        with open(csv_file_path, 'r', encoding='utf-8') as file:
            reader = csv.DictReader(file)
            
            # Kiểm tra header
            required_columns = ['input', 'output']
            if not all(col in reader.fieldnames for col in required_columns):
                result["is_valid"] = False
                result["errors"].append(
                    f"Missing required columns. Expected columns: {required_columns}. Found: {reader.fieldnames}."
                )
                return result
            
            # Kiểm tra nội dung từng dòng và đếm số lượng hợp lệ
            for i, row in enumerate(reader, start=1):
                try:
                    input_value = row['input']
                    output_value = row['output']

                    # Đếm input hợp lệ
                    if input_value and input_value.strip():
                        result["input_count"] += 1

                    # Đếm output hợp lệ
                    if output_value and output_value.strip():
                        result["output_count"] += 1

                    # Kiểm tra nếu dòng thiếu giá trị
                    if not input_value.strip() or not output_value.strip():
                        result["is_valid"] = False
                        result["errors"].append(f"Row {i} is missing data. Input: {input_value}, Output: {output_value}")

                except KeyError as e:
                    result["is_valid"] = False
                    result["errors"].append(f"Row {i} is missing a required field: {str(e)}")
                except Exception as e:
                    result["is_valid"] = False
                    result["errors"].append(f"Error in row {i}: {str(e)}")
    
    except FileNotFoundError:
        result["is_valid"] = False
        result["errors"].append(f"File not found: {csv_file_path}")
    except Exception as e:
        result["is_valid"] = False
        result["errors"].append(f"An error occurred: {str(e)}")
    
    return result

# Kiểm tra file CSV
csv_path = "D:/2023-playwright/train_data.csv"
validation_result = validate_csv_structure_and_count(csv_path)

# In kết quả
if validation_result["is_valid"]:
    print("CSV structure is valid.")
else:
    print("CSV structure is invalid.")
    for error in validation_result["errors"]:
        print(f"Error: {error}")

print(f"Number of valid inputs: {validation_result['input_count']}")
print(f"Number of valid outputs: {validation_result['output_count']}")


CSV structure is valid.
Number of valid inputs: 2826
Number of valid outputs: 2826
